In [1]:
import pandas as pd
import functools
import numpy as np
#from metrics import evaluate
import math
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import uniform
from tqdm import tqdm


import pickle

In [2]:

EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error

    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def _relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1e-10
        else:
            seasonality = benchmark
        return _error(actual[seasonality:], predicted[seasonality:]) /\
               (_error(actual[seasonality:], _naive_forecasting(actual, seasonality)) + EPSILON)

    return _error(actual, predicted) / (_error(actual, benchmark) + EPSILON)


def _bounded_relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Bounded Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark

        abs_err = np.abs(_error(actual[seasonality:], predicted[seasonality:]))
        abs_err_bench = np.abs(_error(actual[seasonality:], _naive_forecasting(actual, seasonality)))
    else:
        abs_err = np.abs(_error(actual, predicted))
        abs_err_bench = np.abs(_error(actual, benchmark))

    return abs_err / (abs_err + abs_err_bench + EPSILON)


def _geometric_mean(a, axis=0, dtype=None):
    """ Geometric mean """
    if not isinstance(a, np.ndarray):  # if not an ndarray object attempt to convert it
        log_a = np.log(np.array(a, dtype=dtype))
    elif dtype:  # Must change the default dtype allowing array type
        if isinstance(a, np.ma.MaskedArray):
            log_a = np.log(np.ma.asarray(a, dtype=dtype))
        else:
            log_a = np.log(np.asarray(a, dtype=dtype))
    else:
        log_a = np.log(a)
    return np.exp(log_a.mean(axis=axis))


def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))


def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))


def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())


def me(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Error """
    return np.mean(_error(actual, predicted))


def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))


mad = mae  # Mean Absolute Deviation (it is the same as MAE)


def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))


def mdae(actual: np.ndarray, predicted: np.ndarray):
    """ Median Absolute Error """
    return np.median(np.abs(_error(actual, predicted)))


def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))


def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error

    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0

    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))


def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))


def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))


def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error

    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))


def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))


def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))


def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))


def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error

    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))


def rmsse(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """ Root Mean Squared Scaled Error """
    q = np.abs(_error(actual, predicted)) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))
    return np.sqrt(np.mean(np.square(q)))


def inrse(actual: np.ndarray, predicted: np.ndarray):
    """ Integral Normalized Root Squared Error """
    return np.sqrt(np.sum(np.square(_error(actual, predicted))) / np.sum(np.square(actual - np.mean(actual))))


def rrse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sqrt(np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual))))


def mre(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Error """
    return np.mean(_relative_error(actual, predicted, benchmark))


def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)


def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mdrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Median Relative Absolute Error """
    return np.median(np.abs(_relative_error(actual, predicted, benchmark)))


def gmrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Geometric Mean Relative Absolute Error """
    return _geometric_mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Bounded Relative Absolute Error """
    return np.mean(_bounded_relative_error(actual, predicted, benchmark))


def umbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Unscaled Mean Bounded Relative Absolute Error """
    __mbrae = mbrae(actual, predicted, benchmark)
    return __mbrae / (1 - __mbrae)


def mda(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - predicted[:-1])).astype(int))


METRICS = {
    'mse': mse,
    'rmse': rmse,
    'nrmse': nrmse,
    'me': me,
    'mae': mae,
    'mad': mad,
    'gmae': gmae,
    'mdae': mdae,
    'mpe': mpe,
    'mape': mape,
    'mdape': mdape,
    'smape': smape,
    'smdape': smdape,
    'maape': maape,
    'mase': mase,
    'std_ae': std_ae,
    'std_ape': std_ape,
    'rmspe': rmspe,
    'rmdspe': rmdspe,
    'rmsse': rmsse,
    'inrse': inrse,
    'rrse': rrse,
    'mre': mre,
    'rae': rae,
    'mrae': mrae,
    'mdrae': mdrae,
    'gmrae': gmrae,
    'mbrae': mbrae,
    'umbrae': umbrae,
    'mda': mda,
}


def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('mae', 'mape', 'rmse','mse', 'smape', 'umbrae')):
    results = {}
    for name in metrics:
        try:
            results[name] = METRICS[name](actual, predicted)
        except Exception as err:
            results[name] = np.nan
            print('Unable to compute metric {0}: {1}'.format(name, err))
    return results


def evaluate_all(actual: np.ndarray, predicted: np.ndarray):
    return evaluate(actual, predicted, metrics=set(METRICS.keys()))

In [3]:
# Decission tree
#distributions_dt = dict(max_depth=list(range(2,16)))
regrt_model = DecisionTreeRegressor(max_depth=5)

# AdaBoost with Decission tree
#distributions_ada = {'estimator__base_estimator__max_depth':list(range(2,16))}
ada_regrt_model = MultiOutputRegressor(AdaBoostRegressor(DecisionTreeRegressor(),
                          n_estimators=300), n_jobs=-1)

ada_regrt_chain_model = RegressorChain(AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),
                          n_estimators=300))

# Gradient boosting
grboost_model = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=300,loss='ls', learning_rate=0.1,
                                                               max_depth=5), n_jobs=-1)
grboost_chain_model = RegressorChain(GradientBoostingRegressor(n_estimators=300,loss='ls', learning_rate=0.1,
                                                               max_depth=5))
# Gaussian process
kernel = DotProduct() + WhiteKernel()
gausspr_model = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel), n_jobs=-1)
gausspr_chain_model = RegressorChain(GaussianProcessRegressor(kernel=kernel))

#Random forest
rand_forest_model = RandomForestRegressor(n_estimators=300,max_depth=5)

#XGBosst
xgb_regr = MultiOutputRegressor(XGBRegressor(n_estimators=300,loss='ls', learning_rate=0.1,
                                                               max_depth=5), n_jobs=-1)

models = {
    "adaboost_tree_regressor": ada_regrt_model,
    #"adaboost_tree_regressor_chain":ada_regrt_chain_model,
    #"gaussian_process_regressor": gausspr_model,
    #"gaussian_process_regressor_chain":gausspr_chain_model,
    "gradient_boost": grboost_model,
    #"gaussian_process_regressor_chain":grboost_chain_model,
    "decision_tree_regressor": regrt_model,
    "random_forest": rand_forest_model,
    "xgb_regressor": xgb_regr
}

models_keys = [  "Árbol de decisión (Adaboost)", 
                 # "Árbol de decisión (Adaboost) en cadena", "Procesos Gaussianos", "Procesos Gaussianos en cadena",
                 # "Gradient Boost",  
               "Árbol de decisión", "Random Forest","XGB Regressor"] #, "XGB Regressor Chain"

In [4]:
def create_propositional_table_dataframe(df,w, target, h):
    columns =[]
    for i in range(w,0,-1):
        columns.extend([s + "_lag"+str(i-1) for s in df.columns])
    for t in target:
        for j in range(h):
            columns.append(t+"_ahead"+str(j+1))
    dataframe = pd.DataFrame(columns=columns)
    return dataframe
    
def create_propositional_table(df, w, h, target):
    columns =[]
    for i in range(w,0,-1):
        columns.extend([s + "_lag"+str(i-1) for s in df.columns])
    for t in target:
        for j in range(h):
            columns.append(t+"_ahead"+str(j+1))
    dataframe = pd.DataFrame(columns=columns)
    
    indexes = []
    for i in tqdm(range((len(df)-w-h+1))):
        window = df.iloc[i:(i+w)]
        row = window.values.reshape(1, len(window.columns)*len(window))
        targets = {}
        for t in target:
            row = np.append(row, df[t].iloc[(i+w):(i+w+h)])
            
        dataframe.loc[i]=row.reshape(1, len(row))[0]
        indexes.append(window.index[-1])
    
    dataframe = dataframe.set_index(pd.Series(indexes))
    
    return dataframe

In [5]:
def calculate_metrics(y_test, y_pred, target, model_name, metrics = ('mae', 'mape', 'rmse','mse')):
    horizons = y_test.columns.values
    index_horizons = np.append(horizons,target+"_mean")
    index = [np.array([model_name for i in range(len(horizons)+1)]), index_horizons]
    metrics = pd.DataFrame(evaluate(y_test, y_pred, metrics=metrics))
    metrics.loc[len(horizons)] = metrics.values.mean(axis=0)
    metrics = metrics.set_index(index)
    
    return metrics

In [6]:
def split_data_generator(df, horizon, window, batch_size, initial_year=2006):
    for i in range(2015 - initial_year + 1):
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler_o3 = MinMaxScaler(feature_range=(0, 1))
        year = initial_year + i
        train = df[df.index.year != year]
        test = df[df.index.year == year]

        data_train = scaler.fit_transform(train.values)
        data_test = scaler.transform(test.values)

        scaler_o3.fit(train.iloc[:, -horizon:].values)

        df_train = pd.DataFrame(data=data_train,
                                columns=train.columns)
        df_test = pd.DataFrame(data=data_test,
                               columns=test.columns)

        validation_row = int(len(df_train) * 0.9)

        df_train = df_train.iloc[:validation_row, :]
        df_validation = df_train.iloc[validation_row:, :]
        df_test = df_test
        yield df_train, df_test, df_validation, scaler, scaler_o3, year

In [7]:
import time
def execute_baseline(X_train, y_train, X_test, y_test, models, target, scaler, year):

    test_metrics_global = None
    y_test = pd.DataFrame(data=scaler.inverse_transform(y_test.values))
    results_global = None
    for name, model in models.items():
        start_time = time.time()
        print("Training "+name+"....")
        model.fit(X_train, y_train)
        pkl_filename = "../results/Baseline/models_with_date/"+target+"_"+str(year)+".pkl"
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)
        test_pred = scaler.inverse_transform(model.predict(X_test))
        
        test_metrics = calculate_metrics(y_test, test_pred, target, name)
        test_metrics['time'] = (time.time() - start_time)/60
        
        if test_metrics_global is None:
            test_metrics_global = test_metrics
        else:
            test_metrics_global = test_metrics_global.append(test_metrics)
        
        test_pred = pd.DataFrame(test_pred, columns=y_test.columns, index=y_test.index)
        results_model = pd.concat({"Real": y_test, "Pred": test_pred}, axis=1, names=["Type", "Horizon"])
        results = pd.concat({name: results_model}, axis=1, names=["Model", "Type", "Horizon"])
        if results_global is None:
            results_global = results
        else:
            results_global = pd.concat([results_global, results], axis=1, join='inner')
        print("--- %s minutes ---" % ((time.time() - start_time)/60))    
        
    return test_metrics_global , results_global
            

In [8]:
def execute(df, window, horizon, target, models):
    for df_train, df_test, df_validation, scaler, scaler_o3, year in split_data_generator(df, horizon, window, 32):
        print("============== "+str(year)+" ==============")
        

        X_train = df_train.iloc[:, :-horizon]
        y_train = df_train.iloc[:, -horizon:]

        X_test = df_test.iloc[:, :-horizon]
        y_test = df_test.iloc[:, -horizon:]

        test_metrics, results = execute_baseline(X_train, y_train, X_test, y_test, models, target, scaler_o3, year)
        #train_metrics.to_pickle("metrics/train_metrics_"+target+"_"+str(year))
        test_metrics.to_pickle("../results/Baseline/metrics_with_date/test_metrics_"+target+"_"+str(year))
        results.to_pickle("../results/Baseline/metrics_with_date/results_"+target+"_"+str(year))

In [9]:
df = pd.read_csv('../data/bermejales_preprositional.csv')

In [10]:
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
df = df.set_index("Unnamed: 0")
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Week'] = df.index.isocalendar().week
df['Hour'] = df.index.hour
columns = df.columns.tolist()
columns = columns[-4:] + columns[:-4]


In [11]:
df = df[[col for col in columns if 'O3' in col or 'PM10' in col or 'TMP' in col or col in ['Month', 'Day', 'Week', 'Hour'] ]]

In [12]:
df

,Month,Day,Week,Hour,BERMEJALES-O3-AT_IN_lag23,BERMEJALES-PM10-AT_IN_lag23,BERMEJALES-TMP Media-AT_IN_lag23,BERMEJALES-O3-AT_IN_lag22,BERMEJALES-PM10-AT_IN_lag22,BERMEJALES-TMP Media-AT_IN_lag22,...,BERMEJALES-O3-AT_IN_ahead15,BERMEJALES-O3-AT_IN_ahead16,BERMEJALES-O3-AT_IN_ahead17,BERMEJALES-O3-AT_IN_ahead18,BERMEJALES-O3-AT_IN_ahead19,BERMEJALES-O3-AT_IN_ahead20,BERMEJALES-O3-AT_IN_ahead21,BERMEJALES-O3-AT_IN_ahead22,BERMEJALES-O3-AT_IN_ahead23,BERMEJALES-O3-AT_IN_ahead24
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2005-01-01 23:00:00,1,1,53,23,2.5000,71.371664,6.0000,2.5000,71.371664,6.0000,...,14.6667,24.3333,32.1667,31.3333,31.5000,13.3333,3.1667,2.1667,5.1667,11.8333
2005-02-01 00:00:00,2,1,5,0,2.5000,71.371664,6.0000,7.5000,52.443333,6.6667,...,24.3333,32.1667,31.3333,31.5000,13.3333,3.1667,2.1667,5.1667,11.8333,20.3333
2005-02-01 01:00:00,2,1,5,1,7.5000,52.443333,6.6667,5.5000,73.713332,6.1667,...,32.1667,31.3333,31.5000,13.3333,3.1667,2.1667,5.1667,11.8333,20.3333,18.6667
2005-02-01 02:00:00,2,1,5,2,5.5000,73.713332,6.1667,4.6667,54.083332,5.3333,...,31.3333,31.5000,13.3333,3.1667,2.1667,5.1667,11.8333,20.3333,18.6667,11.6667
2005-02-01 03:00:00,2,1,5,3,4.6667,54.083332,5.3333,3.6667,57.936662,5.0000,...,31.5000,13.3333,3.1667,2.1667,5.1667,11.8333,20.3333,18.6667,11.6667,8.8333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-30 19:00:00,12,30,53,19,13.6667,43.500000,14.0000,13.1667,83.000000,13.0000,...,22.1667,33.1667,36.5000,42.6667,46.6667,43.6667,43.3333,41.3333,37.0000,36.1667
2015-12-30 20:00:00,12,30,53,20,13.1667,83.000000,13.0000,13.8333,85.833300,12.0000,...,33.1667,36.5000,42.6667,46.6667,43.6667,43.3333,41.3333,37.0000,36.1667,36.6667
2015-12-30 21:00:00,12,30,53,21,13.8333,85.833300,12.0000,14.3333,91.333300,11.8333,...,36.5000,42.6667,46.6667,43.6667,43.3333,41.3333,37.0000,36.1667,36.6667,34.8333


In [ ]:
execute(df, 24, 24, "BERMEJALES-O3-AT_IN", models)

============== 2006 ==============
Training adaboost_tree_regressor....


In [13]:
df_metrics = pd.read_pickle('../results/Baseline/metrics_with_date/test_metrics_BERMEJALES-O3-AT_IN_2015')

In [14]:
pd.options.display.max_rows = 999
df_metrics

mae      mape  \
adaboost_tree_regressor 0                          8.212038  0.285804   
                        1                         11.135739  0.380775   
                        2                         12.534873  0.411652   
                        3                         13.399884  0.457700   
                        4                         14.056665  0.481577   
                        5                         14.442391  0.503857   
                        6                         14.781155  0.508847   
                        7                         14.610892  0.494720   
                        8                         14.801174  0.500189   
                        9                         14.689077  0.499007   
                        10                        15.248344  0.543658   
                        11                        14.930105  0.502948   
                        12                        15.153873  0.535172   
                        13                        15.245408  0.526943   
                        14                        15.335056  0.519481   
                        15                        15.449405  0.551724   
                        16                        15.321157  0.526591   
                        17                        15.515067  0.549236   
                        18                        15.405013  0.535869   
                        19                        15.314948  0.524410   
                        20                        15.553334  0.523026   
                        21                        15.312998  0.536835   
                        22                        15.565517  0.535690   
                        23                        15.527967  0.563653   
                        BERMEJALES-O3-AT_IN_mean  14.480920  0.499974   
gradient_boost          0                          6.385781  0.193298   
                        1                          9.567798  0.306191   
                        2                         11.170408  0.362443   
                        3                         12.098939  0.391655   
                        4                         12.757796  0.412694   
                        5                         13.102163  0.425793   
                        6                         13.307803  0.434995   
                        7                         13.512829  0.442660   
                        8                         13.616045  0.446901   
                        9                         13.745828  0.449194   
                        10                        13.763589  0.450258   
                        11                        13.858405  0.453537   
                        12                        13.958776  0.457992   
                        13                        14.035844  0.465012   
                        14                        14.060981  0.467593   
                        15                        14.144119  0.472285   
                        16                        14.192417  0.475444   
                        17                        14.226650  0.476142   
                        18                        14.277695  0.476319   
                        19                        14.273982  0.479349   
                        20                        14.312153  0.480192   
                        21                        14.271875  0.478612   
                        22                        14.308454  0.480886   
                        23                        14.362564  0.481842   
                        BERMEJALES-O3-AT_IN_mean  13.221371  0.435887   
decision_tree_regressor 0                         14.473017  0.507843   
                        1                         15.953496  0.553417   
                        2                         16.999880  0.598035   
                        3                         17.469564  0.620689   
                        4                